In [1]:
"""
processed_ds = DataProcessor()

subset_trainer = SubsetTrainer(
                                training_args=training_args,
                                valid_ds=processed_ds["valid"]
                                test_ds=processed_ds["test"]
                                )

subset_searcher = SubsetSearcher(
                            subset_trainer=subset_trainer,
                            data_pool=processed_ds[]
                            ...subset args
)

subset_searcher.search()
"""

'\nprocessed_ds = DataProcessor()\n\nsubset_trainer = SubsetTrainer(\n                                training_args=training_args,\n                                valid_ds=processed_ds["valid"]\n                                test_ds=processed_ds["test"]\n                                )\n\nsubset_searcher = SubsetSearcher(\n                            subset_trainer=subset_trainer,\n                            data_pool=processed_ds[]\n                            ...subset args\n)\n\nsubset_searcher.search()\n'

In [2]:
%load_ext autoreload  
%autoreload 2 

In [3]:
import sqlite3
from typing import Any, Optional
from pydantic import BaseModel, Extra, Field
from transformers import TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import numpy as np
import json
import datasets
import wandb
import torch
from tqdm import tqdm
from subset_active_learning.subset_selection import select

/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
training_args = select.SubsetTrainingArguments()
searching_args = select.SubsetSearcherArguments(seed=0)

In [5]:
processed_ds = select.preprocess_sst2(training_args.model_card)

No config specified, defaulting to: sst/default
Reusing dataset sst (/Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 224.53it/s]
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-4d2a8e171c61da81.arrow
100%|██████████| 1101/1101 [00:00<00:00, 1530.56ex/s]
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-ef19ad1c3ea49849.arrow
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-b4a35d41b5184e9b.arrow
100%|██████████| 1101/1101 [00:00<00:00, 3071.34ex/s]
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datase

In [6]:
subset_trainer = select.SubsetTrainer(
    params=training_args, valid_ds=processed_ds["validation"], test_ds=processed_ds["test"]
)

In [7]:
data_pool = processed_ds["train"].shuffle(seed=searching_args.seed).select(range(searching_args.total_sample_size))

Loading cached shuffled indices for dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-c4f88b465eaa05f7.arrow


In [8]:
subset_searcher = select.SubsetSearcher(subset_trainer=subset_trainer, params=searching_args, data_pool=data_pool)


In [9]:
subset_searcher.subset_trainer